In [2]:
import json
import scipy as sp
import scipy.sparse
import matplotlib.pyplot as plt
import joblib
import seaborn as sns
import pandas as pd
import numpy as np
from numpy.random import default_rng


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, r2_score, mean_squared_error
from sklearn.feature_selection import RFE
sns.set()
rng = default_rng()
%matplotlib inline

In [8]:
# hero_selection = sp.sparse.load_npz("../data/hero_selection.npz")
# match_winner = np.load("../data/match_winner.npy")
X = sp.sparse.load_npz("../data/big1_game_features.npz")
y = np.load("../data/big1_match_winners.npy")

y = (y * 2) - 1
print(X.shape, y.shape)

(2162047, 14161) (2162047,)


In [39]:
a = X[0].todense().tolist()
set(a[0])



{0, 1}

In [9]:
y.mean()

-0.00038435797186647656

In [10]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True) 

In [20]:
x_train[0].todense().shape

(1, 14161)

In [11]:
clf = LogisticRegression(random_state=21, max_iter=200).fit(x_train, y_train)
y_h = clf.predict(x_test)
acc = accuracy_score(y_test, y_h)

print(clf.score(x_test, y_test))


0.5737209284398295


In [14]:
joblib.dump(clf, "../models/logreg_iter200_57perc.joblib")

['../models/logreg_iter200_57perc']

In [12]:
print(mean_squared_error(y_test, y_h))

1.7051162862406821


In [38]:
print(r2_score(y_test, y_h))

-0.704981565835056


In [39]:
print(confusion_matrix(y_test, y_h))

[[186240 138310]
 [138159 185906]]


In [40]:
clf.coef_[0, 14]

0.0

In [41]:
y_test_prob = clf.predict_proba(x_test) 
m = y_test_prob.max(axis=1)

# antall matches som classifieren er trygg på.
# Hadde vi valgt ut kun disse gamesa, hadde vi nok hatt bedre acc
(m.mean() > m).sum() 

326404

In [3]:
def create_feature_index():
    heroes_path = "../data/hid_to_rid_dict.json"

    with open(heroes_path, 'r') as fp:
        heroes = json.load(fp)
        heroes = {int(k): int(v) for k, v in heroes.items()} 
        
    feature_set = set()
    for hero_id in heroes.keys():
        feature_set.add(("hero", hero_id))

        for hero_opp in heroes.keys():
            if hero_id == hero_opp:
                continue

            key = [hero_id, hero_opp]
            key.sort()
            feature_set.add(("pair_opp", tuple(key)))

        for hero_same in heroes.keys():
            if hero_id == hero_same:
                continue

            key = [hero_id, hero_same]
            key.sort()

            feature_set.add(("pair_same", tuple(key)))

    print("#feature_set:", len(feature_set))
    features_to_index = {}
    index_to_features = {}
    for k, v in enumerate(feature_set):
        features_to_index[v] = k
        index_to_features[k] = v
    
    return features_to_index, index_to_features
    
features_to_index, index_to_features = create_feature_index()

#feature_set: 14161


In [4]:
features_to_index

{('pair_same', (56, 77)): 0,
 ('pair_same', (44, 62)): 1,
 ('pair_same', (16, 22)): 2,
 ('pair_same', (42, 49)): 3,
 ('pair_same', (10, 23)): 4,
 ('pair_same', (24, 35)): 5,
 ('pair_opp', (104, 111)): 6,
 ('pair_opp', (86, 97)): 7,
 ('pair_same', (72, 109)): 8,
 ('pair_same', (54, 105)): 9,
 ('pair_opp', (62, 94)): 10,
 ('pair_same', (22, 79)): 11,
 ('pair_opp', (50, 79)): 12,
 ('pair_same', (4, 75)): 13,
 ('pair_same', (48, 106)): 14,
 ('pair_same', (36, 91)): 15,
 ('hero', 60): 16,
 ('pair_same', (8, 51)): 17,
 ('pair_same', (26, 116)): 18,
 ('pair_opp', (54, 116)): 19,
 ('pair_opp', (64, 66)): 20,
 ('pair_same', (12, 88)): 21,
 ('pair_opp', (48, 117)): 22,
 ('pair_same', (2, 113)): 23,
 ('pair_opp', (14, 36)): 24,
 ('pair_opp', (8, 62)): 25,
 ('pair_opp', (40, 63)): 26,
 ('pair_opp', (28, 48)): 27,
 ('pair_same', (6, 89)): 28,
 ('pair_opp', (34, 89)): 29,
 ('pair_opp', (0, 8)): 30,
 ('pair_opp', (18, 73)): 31,
 ('pair_opp', (68, 103)): 32,
 ('pair_opp', (22, 49)): 33,
 ('pair_opp', 

In [43]:
logreg = LogisticRegression(random_state=21, max_iter=200)
rfe = RFE(logreg, n_features_to_select=100, step=0.1, verbose=1)
rfe = rfe.fit(x_train, y_train)

for i in range(X.shape[1]):
    print('Column: {}, Selected {}, Rank: {:.3f}'.format(i, rfe.support_[i], rfe.ranking_[i]))

# order = sel.ranking_

# feat_ranks = []
# for i in order:
#     feat_ranks.append(f"{i}: {index_to_features[i]}")
# feat_ranks

Fitting estimator with 14161 features.
Fitting estimator with 12745 features.


/home/jupyter-baoi/.conda/envs/dota/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fitting estimator with 11329 features.
Fitting estimator with 9913 features.
Fitting estimator with 8497 features.
Fitting estimator with 7081 features.
Fitting estimator with 5665 features.
Fitting estimator with 4249 features.
Fitting estimator with 2833 features.
Fitting estimator with 1417 features.
Column: 0, Selected False, Rank: 3.000
Column: 1, Selected False, Rank: 7.000
Column: 2, Selected False, Rank: 6.000
Column: 3, Selected False, Rank: 6.000
Column: 4, Selected False, Rank: 4.000
Column: 5, Selected False, Rank: 7.000
Column: 6, Selected False, Rank: 3.000
Column: 7, Selected False, Rank: 5.000
Column: 8, Selected False, Rank: 3.000
Column: 9, Selected False, Rank: 4.000
Column: 10, Selected False, Rank: 7.000
Column: 11, Selected False, Rank: 6.000
Column: 12, Selected False, Rank: 7.000
Column: 13, Selected False, Rank: 7.000
Column: 14, Selected False, Rank: 7.000
Column: 15, Selected False, Rank: 7.000
Column: 16, Selected False, Rank: 6.000
Column: 17, Selected Fals